In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors

## PART 1. Toronto's neighborhoods data
### 1.1. Scraping data

We scrape the wikipedia page of postal codes of Canada. Our resulting data set is a table with 3 columns: postal code, borough, neigborhood.

In [2]:
# We specify the url of the webpage, we send a request to the server and save the response in a object called "requete".
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
requete=requests.get(url)

In [3]:
# We create a BeautifulSoup object.
page=requete.content
soup=BeautifulSoup(page)

In [4]:
# We print the content of the scraped page.
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XeqLzApAAEAAAFD8CJoAAAAQ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":929562264,"wgRevisionId":929562264,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario

In [5]:
# We search for the useful data that are contained in the table and extract them from the soup.
table = soup.find("table", {"class":"wikitable sortable"})
rows = table.findAll("tr")

In [6]:
# For each row of the table (i.e. for each <tr> except heading table), we assign all the 3 <td> to a variable "cell".
# Then we add the text of each <td> to the appropriate list (postcode, borough and neighbhood) and we create a dataframe.
postcode=[]
borough=[]
neighborhood=[]
for row in rows:
    cell = row.findAll("td")
    if len(cell)==3:
        postcode.append(cell[0].find(text=True))
        borough.append(cell[1].find(text=True))
        neighborhood.append(cell[2].find(text=True))
data=pd.DataFrame({"PostalCode":postcode, "Borough":borough, "Neighborhood":neighborhood})
data.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 1. 2. Preprocessing

Now we clean our raw dataframe: we drop not assigned boroughs, we affect borough's name to not assigned neighborhood, and we group rows for identical postal codes.

In [7]:
print("data shape:", data.shape)

data shape: (287, 3)


In [8]:
# We replace "Not assigned" by "NaN" in "Borough" column and we drop all rows containing NaN.
data.replace("Not assigned", np.nan, inplace = True)
data.dropna(axis=0, inplace=True)
data.reset_index(drop=True, inplace=True)
data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [9]:
# We change the not assigned neighborhoods into the corresponding boroughs.
data[data["Neighborhood"]=="Not assigned\n"]
data["Neighborhood"].replace("Not assigned\n", data["Borough"], inplace=True)
data.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [10]:
print("data shape after cleaning:", data.shape)

data shape after cleaning: (210, 3)


In [11]:
# We group in the same row the neighborhoods that have the same postal code.
# First we notice that some neighborhoods end by "\n", so we delete this character.
data["Neighborhood"].values.tolist()
data["Neighborhood"]=[car[:-1] if car[-1]=="\n" else car for car in data["Neighborhood"].values.tolist()] 
# In order to merge neighborhood cells, we introduce the delimiter "comma" and we group by PostalCode.
data["Neighborhood"]=data["Neighborhood"]+", "
data_new=data.groupby(["PostalCode", "Borough"]).sum()
data_new.reset_index(drop=False, inplace=True)
# We delete the separator "comma" at the end of each neighborhood's cell.
data_new["Neighborhood"]=[car[0:-2] for car in data_new["Neighborhood"]]

In [12]:
data_new.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## PART 2. Geospatial data

We get the latitude et longitude coordinates of each borhood.

In [ ]:
# import geocoder

In [14]:
# We test geocoder module with Toronto's ip.
g = geocoder.ip("199.7.157.0")
print(g.city, g.postal, "->> geocoder ok")

Toronto M5N ->> geocoder ok


In [15]:
# ------------As this module is said unreliable, we try to get back the coordinates of M5N.
#lat_lng_coords = None
#while(lat_lng_coords is None):
#    g=geocoder.google("M5N")
#    lat_lng_coords = g.latlng
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [14]:
# The Geocoder package does not work. So we use the geospatial_data.csv file.
lat_long=pd.read_csv("https://cocl.us/Geospatial_data")
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
# We merge the dataframes lat_long and data_new.
data_f=pd.merge(data_new,lat_long, left_on="PostalCode", right_on="Postal Code", how="inner")
data_f.drop("Postal Code", axis=1, inplace=True)

In [16]:
borough_list=data_f["Borough"].unique()
print("Toronto's boroughs:\n", borough_list)

Toronto's boroughs:
 ['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']


## PART 3. Exploring and clustering Toronto neighborhoods

### 3. 1. Map of Toronto and its neighborhoods.

In [18]:
!conda install -c conda-forge folium=0.5.0 --yes 
print("ok")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [19]:
import folium

In [20]:
# We get the latitude and longitude values of Toronto by using geopy library.
#In order to create an instance of the geocoder, we define a user_agent named "toronto_explorer".
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


In [21]:
# We match each borough with a color.
print("There are {} boroughs in Toronto.".format(borough_list.size))
color_list=["firebrick","salmon","sandybrown","orange","gold","greenyellow","purple","springgreen","turquoise","dodgerblue","navy"]
colors={key: col for key, col in zip(borough_list,color_list)}

There are 11 boroughs in Toronto.


In [22]:
# We create a map of Toronto using latitude and longitude values and we represent each neighborough with the corresponding color.
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(data_f['Latitude'], data_f['Longitude'], data_f['Borough'], data_f['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    col=colors[borough]
    folium.CircleMarker(
       [lat, lng],
       radius=5,
       popup=label,
       fill=True,
       color=col,
       fill_color=col,
       fill_opacity=0.7,
       parse_html=False).add_to(map_toronto)  
map_toronto

### 3.2. Clustering
As the issue is to move on eastern part of the city, we decide to study the neighborhoods colored in yellow, orange, sandybrown, turquoise and greenyellow. So we limit the area to East Toronto, Downtown Toronto, East York, Queen's Park and Central Toronto.

In [23]:
dataclust=data_f[(data_f["Borough"]=="East Toronto") | (data_f["Borough"]=="Downtown Toronto")|(data_f["Borough"]=="East York")|(data_f["Borough"]=="Queen's Park")|(data_f["Borough"]=="Central Toronto")]
print("Number of selected neighborhoods in the studied area: ", len(dataclust))

Number of selected neighborhoods in the studied area:  39


Now, we use the Foursquare API to explore these 39 neighborhoods and segment them.

In [24]:
CLIENT_ID = '' # my Foursquare ID
CLIENT_SECRET = '' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('my credentials ok')

my credentials ok


In [25]:
dataclust.sort_values(by=["Borough"]).shape

(39, 5)

In [26]:
radius=500
LIMIT=100
# We define a function that extracts the category of the 100 venues located at 500 m or less from the neighborhood's center.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # We create the API request URL.
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # We make the GET request.
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # We return only relevant information for each nearby venue.
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

We create a dataframe that contains venues for each neighborhood.

In [27]:
toronto_venues = getNearbyVenues(names=dataclust['Neighborhood'],
                                   latitudes=dataclust['Latitude'],
                                   longitudes=dataclust['Longitude'])

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(1635, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub
1,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
2,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,Shoppers Drug Mart,43.705933,-79.312825,Pharmacy
3,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,TD Canada Trust,43.705740,-79.312270,Bank
4,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,Pizza Pizza,43.705159,-79.313130,Pizza Place


In [29]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 unique categories.


In [30]:
# Now we structure the dataframe before clustering.
# First one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# We add names of neighborhoods at the end of the normalized dataframe.
toronto_onehot["Neighborhoods"]=toronto_venues['Neighborhood']
# We put the "Neighborhoods" column en first position.
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
tor_onehot = toronto_onehot[fixed_columns]
tor_onehot.shape

(1635, 233)

In [328]:
# We visualize our new normalized dataframe.
tor_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Yoga Studio
0,"Woodbine Gardens, Parkview Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Woodbine Gardens, Parkview Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Now we group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.


In [31]:
toronto_grouped = tor_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,0.0,0.010000,0.0,0.03,0.0,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.01,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.03,0.010000,0.0,0.0,0.000000,0.050000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.070000,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.02,0.0,0.000000,0.0,0.0,0.010000,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.020000,0.0,0.000000

We print each neighborhood along with its top 5 most common venues.

In [32]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3  Thai Restaurant  0.04
4           Bakery  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3         Cheese Shop  0.04
4          Steakhouse  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2            Restaurant  0.06
3           Pizza Place  0.06
4  Fast Food Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0      Airport Service  0.18
1     Airport Terminal  0.12
2             Boutique  0.06
3              Airport  0.06
4  Rental Car Location  0.06


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1 

Now we need a new dataframe giving the most common venues for each neigborhood.

In [33]:
# We write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)  
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
# We create a new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# We create columns according to number of top venues.
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# We create the new dataframe.
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# We check the dataframe.
neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Sushi Restaurant,Burger Joint,Bar,Bakery,Asian Restaurant
1,Berczy Park,Coffee Shop,Seafood Restaurant,Beer Bar,Steakhouse,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Café,Restaurant
2,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Coffee Shop
4,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pub,Italian Restaurant,Bakery,Chinese Restaurant,Pizza Place,Outdoor Sculpture,Sandwich Place


We cluster neighborhoods by applying K-Means algorithm.

In [35]:
from sklearn.cluster import KMeans
# We choose the number of clusters.
kclusters = 5
# We delete the first column.
toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)
# We run k-means clustering.
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# We check cluster labels generated for the 10 first rows in the dataframe.
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:
# We add clustering labels in a new column in the dataframe.
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [37]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Sushi Restaurant,Burger Joint,Bar,Bakery,Asian Restaurant
1,0,Berczy Park,Coffee Shop,Seafood Restaurant,Beer Bar,Steakhouse,Cocktail Bar,Bakery,Cheese Shop,Farmers Market,Café,Restaurant
2,0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
3,0,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Coffee Shop
4,0,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pub,Italian Restaurant,Bakery,Chinese Restaurant,Pizza Place,Outdoor Sculpture,Sandwich Place


In [313]:
dataclust.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
35,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
36,M4C,East York,Woodbine Heights,43.695344,-79.318389
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
38,M4G,East York,Leaside,43.709060,-79.363452
39,M4H,East York,Thorncliffe Park,43.705369,-79.349372


In [38]:
# We add latitude/longitude for each neighborhood in the new dataframe, by merging the dataframes "dataclust" and "neighborhoods_venues_sorted".
toronto_merged = pd.merge(dataclust, neighborhoods_venues_sorted,left_on="Neighborhood",right_on="Neighborhoods",how="inner")
toronto_merged.drop("Neighborhoods", axis=1)
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,0,"Woodbine Gardens, Parkview Hill",Fast Food Restaurant,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Pet Store,Bus Line,Bank,Gym / Fitness Center
1,M4C,East York,Woodbine Heights,43.695344,-79.318389,0,Woodbine Heights,Pharmacy,Athletics & Sports,Cosmetics Shop,Curling Ice,Bus Stop,Skating Rink,Spa,Beer Store,Park,Video Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,The Beaches,Neighborhood,Trail,Health Food Store,Pub,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
3,M4G,East York,Leaside,43.709060,-79.363452,0,Leaside,Sporting Goods Shop,Coffee Shop,Burger Joint,Furniture / Home Store,Sushi Restaurant,Beer Store,Brewery,Supermarket,Fish & Chips Shop,Sports Bar
4,M4H,East York,Thorncliffe Park,43.705369,-79.349372,0,Thorncliffe Park,Indian Restaurant,Burger Joint,Yoga Studio,Bank,Grocery Store,Gym,Gas Station,Intersection,Liquor Store,Discount Store


In [39]:
# We check the size of the new dataframe: 39 neighborhoods.
toronto_merged.shape

(39, 17)

We visualize the resulting clusters on the Toronto's map.

In [40]:
from matplotlib.colors import rgb2hex
# We create a map.
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

In [41]:
# We set color scheme for the clusters.
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
colors_array= [rgb2hex(i) for i in colors_array]

# We add markers to the map.
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors_array[cluster-1],
        fill=True,
        fill_color=colors_array[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# We print the map showing neighborhoods' clusters.
map_clusters

Now, we examine each cluster and determine the discriminating venue categories that distinguish each cluster (cf. tables below).
It appears that categories like parks, gardens, playground, or venues like yoga studio, or falafel/ethiopian/dumpling restaurants, are examples of the discriminating venues. So 4 clusters are composed by quiet areas with green zones on top venues and leisure health activities. And one cluster (cluster 0) combines active neighborhoods, with coffee shops, cafés, fast food restaurants on top venues, followed by various facilities such as stores (food stores, beer stores, pet stores...), facilities (banks, pharmacies, restaurants, sport activities...).

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East York,0,"Woodbine Gardens, Parkview Hill",Fast Food Restaurant,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Pet Store,Bus Line,Bank,Gym / Fitness Center
1,East York,0,Woodbine Heights,Pharmacy,Athletics & Sports,Cosmetics Shop,Curling Ice,Bus Stop,Skating Rink,Spa,Beer Store,Park,Video Store
3,East York,0,Leaside,Sporting Goods Shop,Coffee Shop,Burger Joint,Furniture / Home Store,Sushi Restaurant,Beer Store,Brewery,Supermarket,Fish & Chips Shop,Sports Bar
4,East York,0,Thorncliffe Park,Indian Restaurant,Burger Joint,Yoga Studio,Bank,Grocery Store,Gym,Gas Station,Intersection,Liquor Store,Discount Store
6,East Toronto,0,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Restaurant,Furniture / Home Store,Yoga Studio,Fruit & Vegetable Store,Brewery
7,East Toronto,0,"The Beaches West, India Bazaar",Sandwich Place,Pizza Place,Board Shop,Fast Food Restaurant,Steakhouse,Food & Drink Shop,Ice Cream Shop,Liquor Store,Italian Restaurant,Pub
8,East Toronto,0,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Fish Market,Bookstore,Seafood Restaurant,Latin American Restaurant
10,Central Toronto,0,Davisville North,Breakfast Spot,Hotel,Gym,Park,Sandwich Place,Clothing Store,Food & Drink Shop,Discount Store,Dog Run,Doner Restaurant
11,Central Toronto,0,North Toronto West,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Restaurant,Dessert Shop,Burger Joint,Salon / Barbershop,Mexican Restaurant,Diner
12,Central Toronto,0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Gym,Café,Thai Restaurant,Italian Restaurant,Sushi Restaurant,Coffee Shop,Japanese Restaurant


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Central Toronto,1,Roselawn,Garden,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,East York,2,East Toronto,Park,Coffee Shop,Convenience Store,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
15,Downtown Toronto,2,Rosedale,Park,Playground,Trail,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,3,The Beaches,Neighborhood,Trail,Health Food Store,Pub,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Central Toronto,4,Lawrence Park,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
